<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<b>Little Demo: Butler 1</b> <br>
Contact author(s): Melissa Graham <br>
Last verified to run: 2024-06-27 <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Container Size: medium

The `butler` is powerful middleware to query and retrieve LSST data.

This little demo shows how, with user-specified RA, Dec, date-time, and filter, an image and its sources can be retrieved from the `butler` and displayed.

For more detailed `butler` tutorials, see <a href="https://github.com/rubin-dp0/tutorial-notebooks">tutorial notebooks</a> 04a and 04b.

## 1. Set up

Import packages.

In [ ]:
import matplotlib.pyplot as plt
import astropy.time
import numpy as np

import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.sphgeom

Instantiate the `butler` and the butler `registry`.

In [ ]:
butler = dafButler.Butler('dp02', collections='2.2i/runs/DP0.2')
registry = butler.registry

Set the default backend to `matplotlib` and allow images to be displayed in the notebook.

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')
%matplotlib inline

## 2. Define search parameters

User-defined temporal, spatial, and filter search parameters.

In [ ]:
my_ra = 53.60
my_dec = -32.70
my_utctime = "2022-09-15T09:07:20.1"
my_filter = "i"

Define temporal search parameter (`my_timespan`).

In [ ]:
my_datetime = astropy.time.Time(my_utctime, format='isot', scale='utc')
delta_t = astropy.time.TimeDelta(3600, format="sec")
my_timespan = dafButler.Timespan(my_datetime - delta_t, my_datetime + delta_t)

Define spatial search parameter (`htm_id`).

In [ ]:
htm_level=12
pixelization = lsst.sphgeom.HtmPixelization(htm_level)
htm_id = pixelization.index(lsst.sphgeom.UnitVector3d(
    lsst.sphgeom.LonLat.fromDegrees(my_ra, my_dec)))

Determine radius of the spatial search.

In [ ]:
circle = pixelization.triangle(htm_id).getBoundingCircle()
scale = circle.getOpeningAngle().asDegrees()*3600.
print(f'HTM level {htm_level} corresponds to a search radius of ~{scale:0.2f} arcsec.')

## 3. Butler data query

Use the search parameters above to query the butler registry for images at that location in that time span.

In [ ]:
datasetRefs = registry.queryDatasets("calexp", htm20=htm_id,
                                     where="visit.timespan OVERLAPS my_timespan "\
                                     "AND band = '"+my_filter+"'",
                                     bind={"my_timespan": my_timespan})

List the matching `calexp`s (a `calexp` is a processed visit image).

In [ ]:
for i, ref in enumerate(datasetRefs):
    visitInfo = butler.get('calexp.visitInfo', dataId=ref.dataId)
    print(ref.dataId['visit'], ref.dataId['detector'], ref.dataId['band'], 
          visitInfo.date)

Choose one of the above to define the `dataId`.

In [ ]:
my_dataId = {'visit': 192347, 'detector': 68}

Retrieve the `calexp` and its `sources`.

The preferred method to retrieve catalog data is the TAP service, but the butler is convenient for retrieving all sources in a given image.

In [ ]:
my_calexp = butler.get('calexp', dataId=my_dataId)
my_sources = butler.get('src', dataId=my_dataId).asAstropy()

Option to display the `my_sources` table.

In [ ]:
# my_sources

## 4. Plot image with sources

Plot image with sources marked.

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('linear', 'zscale')
display.mtv(my_calexp.image)

with display.Buffering():
    for i in range(len(my_sources)):
        if np.isfinite(my_sources['deblend_psfCenter_x'][i]) \
        & np.isfinite(my_sources['deblend_psfCenter_y'][i]):
            display.dot('o', my_sources['deblend_psfCenter_x'][i], 
                        my_sources['deblend_psfCenter_y'][i], 
                        size=40, ctype='orange')
        
plt.show()

In the above image, some circles appear duplicated and empty.

This is, in part, because no restrictions on columns which, e.g., flag duplicate detections, have been applied to the sources.

## 5. View methods and metadata

Option to print all methods available for a `calexp`.

In [ ]:
# for method in dir(my_calexp):
#     if method[0:1] != '_':
#         print(method)

A few examples of available metadata are provided.

Get the bounding box information (`bbox`), and the center pixels of the `calexp`.

In [ ]:
bbox = my_calexp.getBBox()
print(bbox.centerX, bbox.centerY)

Get the world coordinate system (`wcs`) for the `calexp` and convert its center pixels to sky coordinates.

In [ ]:
wcs = my_calexp.getWcs()
center_coord = wcs.pixelToSky(bbox.centerX, bbox.centerY)
print(center_coord)

Get the weather information for the visit in which this `calexp` was obtained, and print the humidity at the time.

In [ ]:
info = my_calexp.getInfo()
visit_info = info.getVisitInfo()
weather = visit_info.getWeather()
print(weather.getHumidity())

To further explore methods, use `dir` as demonstrated above.

For example, the option to print all methods available for the `bbox`.

In [ ]:
# for method in dir(bbox):
#     if method[0:1] != '_':
#         print(method)